In [1]:
# Code to generate a batch of scenes for the top 50 US cities
import sys
import os
import math
g2sm = "/home/tingjunlab/Development/geo2sigmap/package/src"
sys.path.append(g2sm)
print(sys.path)
from scene_generation.core import Scene

# Top 50 US cities by population with their center coordinates (lat, lon)
TOP_50_CITIES = {
    "New York": (40.7128, -74.0060),
    "Los Angeles": (34.0522, -118.2437),
    "Chicago": (41.8781, -87.6298),
    "Houston": (29.7604, -95.3698),
    "Phoenix": (33.4484, -112.0740),
    "Philadelphia": (39.9526, -75.1652),
    "San Antonio": (29.4241, -98.4936),
    "San Diego": (32.7157, -117.1611),
    "Dallas": (32.7767, -96.7970),
    "San Jose": (37.3382, -121.8863),
    "Austin": (30.2672, -97.7431),
    "Jacksonville": (30.3322, -81.6557),
    "Fort Worth": (32.7555, -97.3308),
    "Columbus": (39.9612, -82.9988),
    "Charlotte": (35.2271, -80.8431),
    "San Francisco": (37.7749, -122.4194),
    "Indianapolis": (39.7684, -86.1581),
    "Seattle": (47.6062, -122.3321),
    "Denver": (39.7392, -104.9903),
    "Washington DC": (38.9072, -77.0369),
    "Boston": (42.3601, -71.0589),
    "El Paso": (31.7619, -106.4850),
    "Nashville": (36.1627, -86.7816),
    "Detroit": (42.3314, -83.0458),
    "Oklahoma City": (35.4676, -97.5164),
    "Portland": (45.5152, -122.6784),
    "Las Vegas": (36.1699, -115.1398),
    "Memphis": (35.1495, -90.0490),
    "Louisville": (38.2527, -85.7585),
    "Baltimore": (39.2904, -76.6122),
    "Milwaukee": (43.0389, -87.9065),
    "Albuquerque": (35.0844, -106.6504),
    "Tucson": (32.2226, -110.9747),
    "Fresno": (36.7378, -119.7871),
    "Mesa": (33.4152, -111.8315),
    "Sacramento": (38.5816, -121.4944),
    "Atlanta": (33.7490, -84.3880),
    "Kansas City": (39.0997, -94.5786),
    "Colorado Springs": (38.8339, -104.8214),
    "Omaha": (41.2565, -95.9345),
    "Raleigh": (35.7796, -78.6382),
    "Miami": (25.7617, -80.1918),
    "Long Beach": (33.7701, -118.1937),
    "Virginia Beach": (36.8529, -75.9780),
    "Oakland": (37.8044, -122.2712),
    "Minneapolis": (44.9778, -93.2650),
    "Tulsa": (36.1540, -95.9928),
    "Tampa": (27.9506, -82.4572),
    "Arlington": (32.7357, -97.1081),
    "New Orleans": (29.9511, -90.0715)
}

def calculate_bounding_box(center_lat, center_lon, width_km=1.0, height_km=1.0):
    """
    Calculate the 4 corner GPS coordinates for a rectangular box around a center point.

    Args:
        center_lat: Center latitude in degrees
        center_lon: Center longitude in degrees
        width_km: Width of the box in kilometers (east-west)
        height_km: Height of the box in kilometers (north-south)

    Returns:
        List of 4 GPS coordinates [lon, lat] for corners: [SW, SE, NE, NW]
        NOTE: Returns coordinates in (longitude, latitude) order for GIS compatibility
    """
    # Earth radius in km
    R = 6371.0

    # Calculate latitude offset (constant regardless of location)
    # 1 degree latitude ≈ 111 km
    lat_offset = (height_km / 2.0) / 111.0

    # Calculate longitude offset (varies with latitude)
    # 1 degree longitude ≈ 111 * cos(latitude) km
    lon_offset = (width_km / 2.0) / (111.0 * math.cos(math.radians(center_lat)))

    # Calculate 4 corners in (lon, lat) order: Southwest, Southeast, Northeast, Northwest
    sw = [center_lon - lon_offset, center_lat - lat_offset]
    se = [center_lon + lon_offset, center_lat - lat_offset]
    ne = [center_lon + lon_offset, center_lat + lat_offset]
    nw = [center_lon - lon_offset, center_lat + lat_offset]

    return [sw, se, ne, nw]

# Generate scenes for each city
for i, (city_name, (lat, lon)) in enumerate(list(TOP_50_CITIES.items())):
    print(f"Generating scene {i+1}/50: {city_name}")

    # Calculate 1 km x 1 km bounding box around city center
    polygon_points_gps = calculate_bounding_box(lat, lon, width_km=1.0, height_km=1.0)

    # Make a new scene instance and save it
    scene_instance = Scene()
    scene_instance(
        points=polygon_points_gps,
        data_dir=f"../scene/scenes/{city_name.lower().replace(' ', '_')}",
        hag_tiff_path=None,
        osm_server_addr="http://10.237.198.210:3452/api/interpreter",
        lidar_calibration=True,
        generate_building_map=True,
        lidar_terrain=True,
        ground_scale=1.2,
        dem_terrain= True
    )

['/home/tingjunlab/anaconda3/envs/g2sm/lib/python312.zip', '/home/tingjunlab/anaconda3/envs/g2sm/lib/python3.12', '/home/tingjunlab/anaconda3/envs/g2sm/lib/python3.12/lib-dynload', '', '/home/tingjunlab/anaconda3/envs/g2sm/lib/python3.12/site-packages', '/home/tingjunlab/Development/geo2sigmap/package/src']
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Generating scene 1/50: New York
Skip the lidar_terrain.ply


Parsing buildings: 100%|██████████| 649/649 [00:04<00:00, 143.86it/s]


Generating scene 2/50: Los Angeles
Skip the lidar_terrain.ply


Parsing buildings: 100%|██████████| 235/235 [00:03<00:00, 66.85it/s] 


Generating scene 3/50: Chicago
Skip the lidar_terrain.ply


...

Parsing buildings: 100%|██████████| 211/211 [00:02<00:00, 73.44it/s] 


Generating scene 48/50: Tampa
Loading local 3DEP dataset polygons...
Done. 3DEP polygons downloaded and projected to  EPSG:32617
Area of Interest: POLYGON ((-9179774.702165356 3242065.8373892508, -9178412.33031288 3242065.8373892508, -9178412.33031288 3243428.2092448846, -9179774.702165356 3243428.2092448846, -9179774.702165356 3242065.8373892508))
Found 1 intersecting datasets
Successfully generated HAG data
Skip the lidar_terrain.ply
FESM files already cached.
[INIT] loading FESM: /home/tingjunlab/.cache/scene_generation/FESM_1m.gpkg
[INIT] loading GRID: /home/tingjunlab/.cache/scene_generation/10_km_cell_grid.gpkg
[CRS] reprojecting FESM from EPSG:4269 to EPSG:4326
[CRS] reprojecting GRID from EPSG:4269 to EPSG:4326
[INIT] spatial indexes ready
No DEM found!
Generating scene 49/50: Arlington
Loading local 3DEP dataset polygons...
Done. 3DEP polygons downloaded and projected to  EPSG:32614
Area of Interest: POLYGON ((-10810739.58576281 3859559.5559953162, -10809308.902041769 3859559.

Parsing buildings: 100%|██████████| 270/270 [00:03<00:00, 88.20it/s] 


Generating scene 50/50: New Orleans
Loading local 3DEP dataset polygons...
Done. 3DEP polygons downloaded and projected to  EPSG:32615
Area of Interest: POLYGON ((-10027407.987503393 3496571.298157253, -10026019.04246929 3496571.298157253, -10026019.04246929 3497960.2431947864, -10027407.987503393 3497960.2431947864, -10027407.987503393 3496571.298157253))
Found 1 intersecting datasets
Successfully generated HAG data
Skip the lidar_terrain.ply
FESM files already cached.
[INIT] loading FESM: /home/tingjunlab/.cache/scene_generation/FESM_1m.gpkg
[INIT] loading GRID: /home/tingjunlab/.cache/scene_generation/10_km_cell_grid.gpkg
[CRS] reprojecting FESM from EPSG:4269 to EPSG:4326
[CRS] reprojecting GRID from EPSG:4269 to EPSG:4326
[INIT] spatial indexes ready


Parsing buildings: 100%|██████████| 565/565 [00:03<00:00, 143.04it/s]


In [2]:
# Test the bounding box calculation for New York
test_lat, test_lon = 40.7128, -74.0060
test_bbox = calculate_bounding_box(test_lat, test_lon, width_km=1.0, height_km=1.0)

print("New York City test:")
print(f"  Center: (lat={test_lat}, lon={test_lon})")
print(f"  Bounding box (4 corners in lon, lat order):")
for i, corner in enumerate(test_bbox):
    print(f"    Corner {i}: [lon={corner[0]:.6f}, lat={corner[1]:.6f}]")
print()

# Check the order and format
print("Corner labels: [SW, SE, NE, NW]")
print(f"  Southwest: lon={test_bbox[0][0]:.6f}, lat={test_bbox[0][1]:.6f}")
print(f"  Southeast: lon={test_bbox[1][0]:.6f}, lat={test_bbox[1][1]:.6f}")
print(f"  Northeast: lon={test_bbox[2][0]:.6f}, lat={test_bbox[2][1]:.6f}")
print(f"  Northwest: lon={test_bbox[3][0]:.6f}, lat={test_bbox[3][1]:.6f}")
print()
print("Verification:")
print(f"  Width (lon):  {abs(test_bbox[1][0] - test_bbox[0][0]):.6f} degrees")
print(f"  Height (lat): {abs(test_bbox[2][1] - test_bbox[1][1]):.6f} degrees")

New York City test:
  Center: (lat=40.7128, lon=-74.006)
  Bounding box (4 corners in lon, lat order):
    Corner 0: [lon=-74.011943, lat=40.708295]
    Corner 1: [lon=-74.000057, lat=40.708295]
    Corner 2: [lon=-74.000057, lat=40.717305]
    Corner 3: [lon=-74.011943, lat=40.717305]

Corner labels: [SW, SE, NE, NW]
  Southwest: lon=-74.011943, lat=40.708295
  Southeast: lon=-74.000057, lat=40.708295
  Northeast: lon=-74.000057, lat=40.717305
  Northwest: lon=-74.011943, lat=40.717305

Verification:
  Width (lon):  0.011885 degrees
  Height (lat): 0.009009 degrees


Creating a custom material
Creating a custom material
Creating a custom material
